# Import Libraries

In [14]:
# Importing libraries
import pandas as pd

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import QuantileTransformer
from sklearn.feature_selection import SelectPercentile , mutual_info_classif

In [3]:
with open('Labels_TargetFeatures.txt') as LabelsName:
    Target = LabelsName.readline().strip()
    Features = LabelsName.readline().split()

# Sub-Model for Dataset by Groups

In [5]:
# Loading Dataset
ObesityDataset_Groups = pd.read_csv('../Exploratory_Data_Analysis/ObesityLevel_Groups.csv',index_col=0)
X_features_groups = ObesityDataset_Groups[Features].to_numpy()
y_target_groups = ObesityDataset_Groups[Target].to_numpy()

In [ ]:
ScoreFunction_MutualInformation = lambda X,y : mutual_info_classif(X,y,n_neighbors=16)

# Model
Pipeline(steps=[('Normalization',QuantileTransformer(output_distribution='normal'))
                ('Feature_Selection',SelectPercentile(ScoreFunction_MutualInformation,percentile=50))])

In [ ]:
# Experimental Phase

## Normal Transformation
X = QuantileTransformer(output_distribution='normal').fit_transform(X_features_groups)

## Feature Selection
ScoreFunction_MutualInformation = lambda X,y : mutual_info_classif(X,y,n_neighbors=16)
Model_Features = SelectPercentile(ScoreFunction_MutualInformation,percentile=50)
X_new = Model_Features.fit_transform(X,y_target_groups)

## Clustering with K-Means

## Clustering with Affinity Propagation

## Clustering with Spectral Clustering

array(['x0', 'x1', 'x2', 'x4', 'x5', 'x6', 'x9', 'x11'], dtype=object)